In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#인지 정상(CN), 경도 인지 장애(MCI) 및 치매(DEM)
file_path = "1.Training/원천데이터/1.걸음걸이/train_activity.csv"
Walk = pd.read_csv(file_path)
file_path = "1.Training/라벨링데이터/1.걸음걸이/training_label.csv"
Walk_label = pd.read_csv(file_path)
Walk_label.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
Walk = pd.merge(Walk, Walk_label[['EMAIL', 'DIAG_NM']], on='EMAIL', how='left')
Walk['EMAIL'] = Walk['EMAIL'].str.extract(r'(\d{3})')
#MET =휴식하고 있을 때 필요한 에너지나 몸에서 필요로 하는 산소의 양
Walk.rename(columns={
    'activity_average_met': '하루간 평균 MET',
    'activity_cal_active': '하루간 활동 칼로리',
    'activity_cal_total': '하루간 총 사용 칼로리',
    'activity_class_5min': '하루간 5분당 활동 로그',
    'activity_daily_movement': '매일 움직인 거리',
    'activity_day_end': '활동 종료 시간',
    'activity_day_start': '활동 시작 시간',
    'activity_high': '고강도 활동 시간',
    'activity_inactive': '비활동 시간',
    'activity_inactivity_alerts': '비활동 알람 횟수',
    'activity_low': '저강도 활동 시간',
    'activity_medium': '중강도 활동 시간',
    'activity_met_1min': '하루간 1분 당 MET 로그',
    'activity_met_min_high': '하루간 고강도 활동 MET',
    'activity_met_min_inactive': '하루간 비활동 MET',
    'activity_met_min_low': '하루간 저강도 활동 MET',
    'activity_met_min_medium': '하루간 중강도 활동 MET',
    'activity_non_wear': '미착용 시간',
    'activity_rest': '휴식 시간',
    'activity_score': '활동 점수',
    'activity_score_meet_daily_targets': '활동 목표달성 점수',
    'activity_score_move_every_hour': '매 시간 당 활동유지 점수',
    'activity_score_recovery_time': '회복시간 점수',
    'activity_score_stay_active': '활동 유지 점수',
    'activity_score_training_frequency': '운동 빈도 점수',
    'activity_score_training_volume': '운동 점수',
    'activity_steps': '매일 걸음 수',
    'activity_total': '활동 총 시간(분)'
}, inplace=True)
Walk['활동 시작 시간'] = pd.to_datetime(Walk['활동 시작 시간'])
Walk['활동 종료 시간'] = pd.to_datetime(Walk['활동 종료 시간'])
Walk['활동시간'] = Walk['활동 종료 시간'] - Walk['활동 시작 시간']

Walk.drop(['활동 시작 시간', '활동 종료 시간','활동시간'], axis=1, inplace=True)
Walk['하루간 5분당 활동 로그'] = Walk['CONVERT(activity_class_5min USING utf8)'].apply(lambda x: len(x.split('/')))
Walk['하루간 1분 당 MET 로그'] = Walk['CONVERT(activity_met_1min USING utf8)'].apply(lambda x: len(str(x).split('/')))
Walk.drop(['CONVERT(activity_class_5min USING utf8)','CONVERT(activity_met_1min USING utf8)'], axis=1, inplace=True)
Walk['DIAG_NM'] = Walk['DIAG_NM'].replace({'CN': 0, 'MCI': 1, 'Dem': 2}).astype(int)
Walk.drop(["EMAIL"], axis=1, inplace=True)



walk_data = Walk.drop(columns=['DIAG_NM'])
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(walk_data)

Walk_nom = pd.DataFrame(normalized_data, columns=walk_data.columns)
Walk_nom['DIAG_NM'] = Walk['DIAG_NM'].reset_index(drop=True)
Walk_nom

,하루간 평균 MET,하루간 활동 칼로리,하루간 총 사용 칼로리,하루간 5분당 활동 로그,매일 움직인 거리,고강도 활동 시간,비활동 시간,비활동 알람 횟수,저강도 활동 시간,중강도 활동 시간,...,활동 점수,활동 목표달성 점수,매 시간 당 활동유지 점수,회복시간 점수,활동 유지 점수,운동 빈도 점수,운동 점수,매일 걸음 수,활동 총 시간(분),DIAG_NM
0,0.307087,0.074242,0.232132,1.000000,0.071862,0.000000,0.659574,0.000000,0.233449,0.016575,...,0.87,0.78,1.00,1.0,0.54,1.00,0.97,0.093113,0.219979,1
1,0.299213,0.054924,0.204505,1.000000,0.053923,0.000000,0.664200,0.000000,0.152149,0.024862,...,0.83,0.60,1.00,1.0,0.58,0.96,0.96,0.072888,0.148778,1
2,0.291339,0.044697,0.198799,1.000000,0.036777,0.000000,0.654024,0.166667,0.145180,0.016575,...,0.73,0.43,0.95,1.0,0.55,0.71,0.83,0.056011,0.139214,1
3,0.307087,0.068182,0.228829,1.000000,0.059817,0.000000,0.651249,0.000000,0.235772,0.013812,...,0.70,0.43,1.00,1.0,0.58,0.71,0.59,0.088581,0.221041,1
4,0.354331,0.141667,0.324625,1.000000,0.115583,0.000000,0.403330,0.000000,0.593496,0.016575,...,0.70,0.43,1.00,1.0,0.89,0.40,0.46,0.179484,0.549416,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,0.322835,0.094318,0.262162,1.000000,0.094537,0.006711,0.720629,0.666667,0.193961,0.060773,...,0.61,0.25,0.43,1.0,0.45,0.40,0.95,0.110079,0.201913,0
9701,0.370079,0.215909,0.361562,1.000000,0.236975,0.006711,0.478261,0.166667,0.385598,0.168508,...,0.78,0.43,0.95,1.0,0.76,0.71,0.93,0.264248,0.418704,0
9702,0.322835,0.111742,0.256156,1.000000,0.110054,0.026846,0.533765,0.000000,0.200929,0.082873,...,0.70,0.25,1.00,1.0,0.70,0.40,0.95,0.127492,0.219979,0
9703,0.314961,0.070455,0.240841,1.000000,0.068111,0.000000,0.718779,0.166667,0.160279,0.049724,...,0.66,0.25,0.95,1.0,0.47,0.40,0.93,0.085076,0.165781,0


In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# CSV 파일 경로 설정
file_path = '1.Training/원천데이터/2.수면/train_sleep.csv' 
# CSV 파일을 판다스 데이터프레임으로 불러오기
Sleep = pd.read_csv(file_path)

# CSV 파일 경로 설정
file_path = '1.Training/라벨링데이터/2.수면/training_label.csv' 
# CSV 파일을 판다스 데이터프레임으로 불러오기
Sleep_label = pd.read_csv(file_path)


Sleep_label.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
Sleep = pd.merge(Sleep, Sleep_label[['EMAIL', 'DIAG_NM']], on='EMAIL', how='left')
Sleep['EMAIL'] = Sleep['EMAIL'].str.extract(r'(\d{3})')

Sleep = Sleep.drop(['sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min', 'sleep_is_longest',
                    'CONVERT(sleep_hr_5min USING utf8)',
                    'CONVERT(sleep_hypnogram_5min USING utf8)',
                    'CONVERT(sleep_rmssd_5min USING utf8)'
                    ], axis=1)

Sleep.rename(columns={
    'EMAIL': '이메일',
    'sleep_awake': '깨어있음',
    'sleep_bedtime_end': '종료시간',
    'sleep_bedtime_start': '시작시간',
    'sleep_breath_average': '평균호흡',
    'sleep_deep': '깊은수면',
    'sleep_duration': '총시간',
    'sleep_efficiency': '효율',
    'sleep_hr_average': '평균심박수',
    'sleep_hr_lowest': '최저심박수',
    'sleep_light': '얕은수면',
    'sleep_midpoint_at_delta': '중간점변화',
    'sleep_midpoint_time': '중간점시간',
    'sleep_onset_latency': '잠들기까지시간',
    'sleep_period_id': '기간_ID',
    'sleep_rem': '렘수면',
    'sleep_restless': '뒤척임',
    'sleep_rmssd': '심박변동성_RMSSD',
    'sleep_score': '점수',
    'sleep_score_alignment': '정렬점수',
    'sleep_score_deep': '깊은수면점수',
    'sleep_score_disturbances': '방해점수',
    'sleep_score_efficiency': '효율점수',
    'sleep_score_latency': '잠들기점수',
    'sleep_score_rem': '렘수면점수',
    'sleep_score_total': '총점',
    'sleep_temperature_delta': '체온변화',
    'sleep_temperature_deviation': '체온편차',
    'sleep_total': '총수면',
}, inplace=True)


# 불필요한 공백이 있을 경우 제거
Sleep.columns = Sleep.columns.str.strip()

Sleep['시작시간'] = pd.to_datetime(Sleep['시작시간'])
Sleep['종료시간'] = pd.to_datetime(Sleep['종료시간'])
Sleep['활동시간'] = Sleep['종료시간'] - Sleep['시작시간']

Sleep.drop(['시작시간', '종료시간','활동시간'], axis=1, inplace=True)
Sleep['DIAG_NM'] = Sleep['DIAG_NM'].replace({'CN': 0, 'MCI': 1, 'Dem': 2})
Sleep.drop(["이메일"], axis=1, inplace=True)


Sleep_data = Sleep.drop(columns=['DIAG_NM'])
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(Sleep_data)

Sleep_nom = pd.DataFrame(normalized_data, columns=Sleep_data.columns)
Sleep_nom['DIAG_NM'] = Sleep['DIAG_NM'].reset_index(drop=True)
Sleep_nom



,깨어있음,평균호흡,깊은수면,총시간,효율,평균심박수,최저심박수,얕은수면,중간점변화,중간점시간,...,깊은수면점수,방해점수,효율점수,잠들기점수,렘수면점수,총점,체온변화,체온편차,총수면,DIAG_NM
0,0.311288,0.583333,0.656920,0.627778,0.637931,0.267436,0.275862,0.376942,0.873681,0.456522,...,1.000000,0.752941,0.619565,0.897959,0.555556,0.930233,0.386719,0.386719,0.556590,1
1,0.230331,0.595238,0.483431,0.413889,0.637931,0.264004,0.275862,0.281276,0.128017,0.304348,...,0.979798,0.741176,0.619565,0.979592,0.424242,0.593023,0.453125,0.453125,0.386819,1
2,0.380844,0.595238,0.300195,0.491667,0.465517,0.251365,0.241379,0.295993,0.151182,0.474480,...,0.939394,0.541176,0.358696,0.755102,0.676768,0.558140,0.373047,0.373047,0.372493,1
3,0.135690,0.547619,0.483431,0.338889,0.758621,0.236698,0.224138,0.206051,0.139098,0.288280,...,0.979798,0.776471,0.793478,0.836735,0.727273,0.558140,0.359375,0.359375,0.368911,1
4,0.253136,0.583333,0.280702,0.229167,0.431034,0.232486,0.241379,0.162715,0.064890,0.206049,...,0.878788,0.682353,0.336957,0.785714,0.242424,0.174419,0.289062,0.289062,0.181948,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,0.208666,0.392857,0.111111,0.522222,0.724138,0.434545,0.413793,0.485691,0.159896,0.420605,...,0.333333,0.505882,0.739130,0.816327,0.949495,0.837209,0.369141,0.369141,0.512178,0
9701,0.197263,0.416667,0.000000,0.391667,0.672414,0.491496,0.482759,0.399019,0.179497,0.338374,...,0.000000,0.494118,0.663043,0.867347,0.888889,0.581395,0.466797,0.466797,0.384670,0
9702,0.173318,0.369048,0.031189,0.570833,0.793103,0.451553,0.465517,0.563369,0.184147,0.446125,...,0.080808,0.529412,0.847826,0.897959,0.979798,0.965116,0.439453,0.439453,0.584527,0
9703,0.058153,0.392857,0.144250,0.394444,0.913793,0.486035,0.465517,0.446443,0.127674,0.337429,...,0.434343,0.529412,0.978261,0.897959,0.858586,0.767442,0.376953,0.376953,0.474928,0


In [17]:
combined_df = pd.concat([Walk_nom, Sleep_nom], axis=1)

def merge_columns(df1, df2):
    for column in df1.columns.intersection(df2.columns):
        if (df1[column] == df2[column]).all():
            df2.drop(column, axis=1, inplace=True)
    return pd.concat([df1, df2], axis=1) 

Dem = merge_columns(Walk_nom, Sleep_nom)
Dem


,하루간 평균 MET,하루간 활동 칼로리,하루간 총 사용 칼로리,하루간 5분당 활동 로그,매일 움직인 거리,고강도 활동 시간,비활동 시간,비활동 알람 횟수,저강도 활동 시간,중강도 활동 시간,...,정렬점수,깊은수면점수,방해점수,효율점수,잠들기점수,렘수면점수,총점,체온변화,체온편차,총수면
0,0.307087,0.074242,0.232132,1.000000,0.071862,0.000000,0.659574,0.000000,0.233449,0.016575,...,0.868687,1.000000,0.752941,0.619565,0.897959,0.555556,0.930233,0.386719,0.386719,0.556590
1,0.299213,0.054924,0.204505,1.000000,0.053923,0.000000,0.664200,0.000000,0.152149,0.024862,...,1.000000,0.979798,0.741176,0.619565,0.979592,0.424242,0.593023,0.453125,0.453125,0.386819
2,0.291339,0.044697,0.198799,1.000000,0.036777,0.000000,0.654024,0.166667,0.145180,0.016575,...,1.000000,0.939394,0.541176,0.358696,0.755102,0.676768,0.558140,0.373047,0.373047,0.372493
3,0.307087,0.068182,0.228829,1.000000,0.059817,0.000000,0.651249,0.000000,0.235772,0.013812,...,1.000000,0.979798,0.776471,0.793478,0.836735,0.727273,0.558140,0.359375,0.359375,0.368911
4,0.354331,0.141667,0.324625,1.000000,0.115583,0.000000,0.403330,0.000000,0.593496,0.016575,...,0.797980,0.878788,0.682353,0.336957,0.785714,0.242424,0.174419,0.289062,0.289062,0.181948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,0.322835,0.094318,0.262162,1.000000,0.094537,0.006711,0.720629,0.666667,0.193961,0.060773,...,1.000000,0.333333,0.505882,0.739130,0.816327,0.949495,0.837209,0.369141,0.369141,0.512178
9701,0.370079,0.215909,0.361562,1.000000,0.236975,0.006711,0.478261,0.166667,0.385598,0.168508,...,1.000000,0.000000,0.494118,0.663043,0.867347,0.888889,0.581395,0.466797,0.466797,0.384670
9702,0.322835,0.111742,0.256156,1.000000,0.110054,0.026846,0.533765,0.000000,0.200929,0.082873,...,1.000000,0.080808,0.529412,0.847826,0.897959,0.979798,0.965116,0.439453,0.439453,0.584527
9703,0.314961,0.070455,0.240841,1.000000,0.068111,0.000000,0.718779,0.166667,0.160279,0.049724,...,1.000000,0.434343,0.529412,0.978261,0.897959,0.858586,0.767442,0.376953,0.376953,0.474928
